In [ ]:
import Coinex_API_Class as CoinexAPI

%reload_ext autoreload
%autoreload 2
    
api_key = "47D7C4B286224298BB3D88A9D7161A45"
api_secret = "F0F2D4156F6892ED2F03576FEE74CFA658C5F8271DCE102D"
coinex = CoinexAPI.Coinex_API(api_key,api_secret)

In [2]:
import json
import sqlite3
import Coinex_API_Class as Coinex

try :
    with open('config.json', 'r') as f:
        data = json.load(f)
        api_key = data['api_key']
        api_secret = data['api_secret']
        db_name = data['db_name']
        num_candles = data['num_candle']
        loss_limit = data['loss_limit']
        client_id = data['client_id']
except Exception as e:
    print(f"Error: {e}")

conn = sqlite3.connect("crypto.db")
coinex = Coinex.Coinex_API(api_key, api_secret, connection=conn , client_id=client_id)

In [3]:
float(coinex.get_spot_price_ticker("FLUXUSDT")[0]["last"])

0.5215